In [ ]:
import os
import json
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
JSON_FILE_PATH = '../data/data.json'
IMAGES_FOLDER_PATH = '../data/images'
LABELLED_IMAGES_FOLDER_PATH = '../data/labelled_images'

OCCLUDED_COLOR = 'red'
NOT_OCCLUDED_COLOR = 'blue'

In [ ]:
def get_dataframe_from_json(json_file):
    """
    Get a pandas DataFrame from a json file with LED coordinates for each frame.
    
    Args:
        json_file (str): Path to the json file.
        
    Returns:
        df (pd.DataFrame): DataFrame with columns ['frame', 'led', 'x', 'y', 'occluded'].
    """
    with open(json_file, 'r') as file:
        data = json.load(file)
    
    frames = []
    for frame, leds in data.items():
        for led, led_data in leds.items():
            frames.append([frame, led, led_data["x_cam"], led_data["y_cam"], led_data["occluded"]])

    df = pd.DataFrame(frames, columns=['frame', 'led', 'x_cam', 'y_cam', 'occluded'])
    df.set_index(['frame', 'led'], inplace=True)
    return df

df = get_dataframe_from_json(JSON_FILE_PATH)
df.head()

In [ ]:
for frame, frame_data in tqdm(df.groupby(level="frame")):
    # Get frame image
    image_path = os.path.join(IMAGES_FOLDER_PATH, f'{int(frame):04d}.png')
    image = mpimg.imread(image_path)
    image_width = image.shape[1]
    image_height = image.shape[0]
    
    # Plot image
    plt.figure(figsize=(image_width / 100, image_height / 100))
    plt.imshow(image, cmap='gray')
    
    # Plot LEDs
    for index, coords in frame_data.iterrows():
        x = coords['x_cam'] * image_width
        y = (1 - coords['y_cam']) * image_height

        if x < 0 or x > image_width or y < 0 or y > image_height:
            print(f"Element at frame {frame} is not visible.")
            continue
        
        # Plot LED
        plt.scatter(x, y, 
                    marker='x', s=20, linewidths=0.7,
                    alpha=0.8, color=OCCLUDED_COLOR if coords['occluded'] else NOT_OCCLUDED_COLOR)
        
    plt.axis('off')
    plt.gca().set_axis_off()
    plt.subplots_adjust(top=1, bottom=0, right=1, left=0, hspace=0, wspace=0)
    plt.margins(0, 0)
    plt.gca().xaxis.set_major_locator(plt.NullLocator())
    plt.gca().yaxis.set_major_locator(plt.NullLocator())
    
    # Save image
    plt.savefig(os.path.join(LABELLED_IMAGES_FOLDER_PATH, f'{int(frame):04d}.png'), dpi=300)
    plt.close()